# Signal Synchronization


In [ ]:
import numpy as np
try:
    import cupy as cp
    NO_CUPY = False
except ImportError:
    NO_CUPY = True
from scipy import signal
import matplotlib.pyplot as plt

from datetime import datetime

import sys
sys.path.append('../')
import pluto_sdr_pr.ioutils
import pluto_sdr_pr.processing
import pluto_sdr_pr.signal

## Correlation-based Synchronization

Use timestamp information (if available) and cross-correlation to find sample offset in recordings. This approach assumes the receiver hardware is phase and frequency coherent (i.e. sharing a common local oscillator).

First some setup-work, which includes reading file headers and determining a rough offset based on recording timestamps.


In [ ]:
CPI = 0.2 # in seconds
start_skip_time = 3

ref_file_path = "../data/pluto_a_ref.2021-08-03T17_49_01_994.sdriq"
surv_file_path = "../data/pluto_b_surv.2021-08-03T17_49_05_196.sdriq"

_, ref_hdr = pluto_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, 0, 0)
_, surv_hdr = pluto_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, 0, 0)

assert ref_hdr["sample_rate"] == surv_hdr["sample_rate"]
assert ref_hdr["center_frequency"] == surv_hdr["center_frequency"]

sample_rate = ref_hdr["sample_rate"]
center_frequency = ref_hdr['center_frequency']
num_samples_in_cpi = CPI * sample_rate

print(f"Sample rate: {sample_rate / 1e6:0.2f} MHz, Center frequency: {center_frequency / 1e6:0.1f} MHz")
print(f"Start of reference channel: {ref_hdr['start']}")
print(f"Start of surveillance channel: {surv_hdr['start']}")

time_diff = surv_hdr['start'] - ref_hdr['start']
estim_sample_shift = time_diff.seconds * sample_rate

print(f"Time delta: {time_diff}, Estimated sample shift: {estim_sample_shift}")

ref_samples, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, int(num_samples_in_cpi * 2), start_skip_time * sample_rate + max(0, int(estim_sample_shift - num_samples_in_cpi)))
surv_samples, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, int(num_samples_in_cpi * 2), start_skip_time * sample_rate + max(0, -int(estim_sample_shift - num_samples_in_cpi)))

sub_sample_factor = 1
ref_samples = ref_samples[::sub_sample_factor]
surv_samples = surv_samples[::sub_sample_factor]

Now correlate and find the correlation peak.


In [ ]:
NO_CUPY = True
if NO_CUPY:
    corr = signal.correlate(ref_samples, surv_samples, mode="same", method="fft")
else:
    corr = cp.correlate(cp.asarray(ref_samples), cp.asarray(surv_samples), mode="same").get()
corr_mag = np.abs(corr)
corr_mag_norm = corr_mag / np.max(corr_mag)

peak_idx = np.argmax(corr_mag_norm)
peak_time_offset = (peak_idx - corr_mag_norm.size / 2) / sample_rate * sub_sample_factor

print(f"Correlation peak at {peak_idx} with time offset of {peak_time_offset * 1000:0.3f} msec after initial estimation")

In [ ]:
plt.figure(figsize=(40, 7))
plt.plot(np.linspace(-corr_mag_norm.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_norm.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_norm.size), corr_mag_norm)
plt.xlabel("time delta [msec]")
plt.ylabel("correlation [dB]")
plt.xticks(np.arange(-CPI * 1000, CPI * 1000 + 1, step=100, dtype=np.int32))
plt.grid()

plt.annotate(f"Peak\n"
             f"sample offset: {peak_idx * sub_sample_factor}\n"
             f"time offset: {peak_time_offset * 1000:0.3f} msec",
             xy=(peak_time_offset * 1000, corr_mag_norm[peak_idx]),
             xytext=(-100, 30),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'});

## PSS Synchronization

5G Broadcast communication uses two types of signals to allow a UE to synchronize with the eNodeB. The Primary Synchronization Signal (PSS) is one of three Zadoff-Chu sequences given by

$$
  d_u(n) = \begin{cases}
             \mathrm{exp}\left(-\mathrm{j} \frac{\pi u n (n + 1)}{63}\right) & n = 0, 1, \dots, 30 \\
             \mathrm{exp}\left(-\mathrm{j} \frac{\pi u n (n + 1)(n + 2)}{63}\right) & n = 31, 32,\dots, 61 \\
           \end{cases}
$$

where the Zadoff-Chu root sequence index $u$ is given by the following table

|Physical Layer Identity $N^{(2)}_\mathrm{ID}$|Root Index $u$|
|---------------------|--------------|
|$0$|$25$|
|$1$|$29$|
|$2$|$34$|

We start by generating the Zadoff-Chu sequences mentioned above.


In [ ]:
z = pluto_sdr_pr.signal.generate_pss_sequence([0, 1, 2])

Test the generated sequences by calculating their auto-correlation. A neat property of Zadoff-Chu sequences is that their auto-correlation is $\approx 0$ for all non-zero lag terms.


In [ ]:
print(f"First 5 elements of each Zadoff-Chu sequence\n{z[:, :5]}")

corr_abs = np.abs(signal.correlate(z[0, :], z[0, :], mode='full', method="fft"))

plt.figure(figsize=(15, 5))
plt.plot(np.arange(-corr_abs.shape[0] // 2 + 1, corr_abs.shape[0] // 2 + 1), corr_abs / np.max(corr_abs))
plt.xlabel("lag [symbols]")
plt.ylabel("norm. correlation")
plt.title("auto-correlation of first Zadoff-Chu sequence ($u = 25$)")
plt.grid()

In [ ]:
freqz = np.abs(np.fft.fftshift(np.fft.fft(ref_samples[:4096])))
signal_bw = 5e6

plt.figure(figsize=(20, 5))
plt.plot(np.linspace((center_frequency - sample_rate / 2) * 1e-6, (center_frequency +  sample_rate / 2) * 1e-6, 4096), freqz)
plt.title("Frequency Spectrum of Observation Signal")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Amplitude")
plt.grid()
plt.axvspan((center_frequency - signal_bw / 2) * 1e-6, (center_frequency + signal_bw / 2) * 1e-6, color="red", alpha=0.2);

### PSS Reference Subframe Generation

Derive modulation parameters based on some constants.


In [ ]:
num_dl_rb = int(0.9 * signal_bw / 180e3)
sc_per_rb = 12
num_dl_sc = num_dl_rb * sc_per_rb
print(f"Assuming {num_dl_rb:d} downlink resource-blocks given channel bandwidth of {signal_bw * 1e-6:.1f} MHz")

symbols_per_slot = 7
slots_per_subframe = 2
symbols_per_subframe = symbols_per_slot * slots_per_subframe

num_pss_rb = 5 # PSS is always located in the middle 5 RBs
num_sss_rb = 5 # SSS is always located in the middle 5 RBs
pss_symbol_idx_in_slot = 6 # assuming 0,1,...,6 symbols per slot
pss_sc_offset_in_symbol = int(((num_dl_rb - num_pss_rb) / 2) * sc_per_rb) # sub-carrier offset within symbol that points to the start of PSS

sss_symbol_idx_in_slot = 4
sss_sc_offset_in_symbol = int(((num_dl_rb - num_sss_rb) / 2) * sc_per_rb) # sub-carrier offset within symbol that points to the start of SSS

We will now construct a reference subframe containing only the PSS sequence located in the middle 5 RBs.


In [ ]:
grid = np.zeros((z.shape[0], num_dl_sc, symbols_per_subframe), dtype=np.complex128)

grid[:, np.arange(pss_sc_offset_in_symbol, pss_sc_offset_in_symbol + z.shape[1]), pss_symbol_idx_in_slot] = z[:, :]

pss_waveforms, gen_sample_rate = pluto_sdr_pr.signal.ofdm_modulate_subframe(grid)

Our cell search works by generating a subframe containing only the PSS or SSS (first PSS then in a second run SSS). These reference subframes are then sequentially cross-correlated across the input channel data, marking the PSS/SSS (aka cell-id) combinations with the highest peaks. After going through PSS/SSS combinations, the one with the highest peak is determined the correct cell-id.
In the block above we generated a grid of resource blocks matching the input data. The grid is filled with either a Zadoff-Chu- (PSS) or m-sequence (SSS) in frequency domain. To make the correlation, the data must be OFDM modulated into a time-domain signal. Like any practical OFDM implementation LTE uses a cyclic prefix (to alleviate issues with multi-path reception). That is, each data point is converted into an OFDM symbol by means of Inverse Fourier Transform (IFT), then a specific amount of samples from the end of each symbol (after IFT!) are copied to the front of each symbol respectively. The quantity of copied samples depends on the position of the symbol inside the slot as well as channel settings (normal or extended cyclic prefix, sub-carrier spacing, etc.). For normal cyclic prefix the sum of all CPs in a subframe is 2048 for a 20MHz channel.

$$
N_{\text{CP}, l} = \begin{cases}
160 & \mathrm{mod}(l, 7) = 0 \\
144 & \mathrm{mod}(l, 7) = 1, 2, \dots, 6 \\
\end{cases}
$$

For smaller channels this calculation is scalled down in steps of powers of two (1024, 512, 256, ...), which ensures that  is always an integer number of samples. Like in the case of 5MHz, 25 RBs equating to 300 sub-carriers (12 SC per 1 RB) are available. The closest power of two after 300 would be 512. The 300 SCs are copied to the center IFT bins, padding the rest with zeros. This now virtually 512 SC wide symbol results in a larger required sample count per slot -> subframe -> frame and finally samples per second.

Numer of samples prefixed per slot:
$$
N_{\text{CP}, \text{slot}} = 512
$$
Number of samples per symbol, i.e. size of FFT:
$$
N_{\text{sym}} = 512
$$
Resulting number of samples per second:
$$
N = \left( N_{\text{CP}, \mathrm{slot}} + \underset{\text{symbols per subframe}}{14} \cdot N_{\mathrm{sym}} \right) \cdot \underset{\text{subframes per frame}}{10} \cdot \underset{\text{frames per sec}}{100} = 7.68 \cdot 10^6
$$

Thus modulation occurs at a much higher sampling rate of 7.68MHz than necessary.
Now the sample rate of our recording might be lower than that (because the physical channel may only be 5MHz wide), but we have to artificially widen it to match the 7.68MHz of our reference subframe. This is done here, by simply padding with zeros in frequency domain.


In [ ]:
T_S = 1 / (15000 * 2048)
T_FRAME = T_S * 307200

frame_count = 20

sample_rate_diff = gen_sample_rate - sample_rate

obsrv_samples = ref_samples[:int(sample_rate * T_FRAME * frame_count)]

if sample_rate_diff > 0:
    obsrv_freqs = np.fft.fftshift(np.fft.fft(obsrv_samples))
    padded_obsrv_fftout = np.pad(obsrv_freqs, pad_width=(int(sample_rate_diff * T_FRAME * frame_count) // 2, int(sample_rate_diff * T_FRAME * frame_count) // 2))
    obsrv_samples = np.fft.ifft(np.fft.fftshift(padded_obsrv_fftout))

    plt.figure(figsize=(20,7))
    plt.plot(np.linspace(-gen_sample_rate / 2, gen_sample_rate / 2, padded_obsrv_fftout.shape[0]) * 1e-6, np.abs(padded_obsrv_fftout))
    plt.title("Frequency Spectrum of (padded) Observation Signal")
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Amplitude")
    closest_500_khz_divisor = (gen_sample_rate / 2 - np.mod(gen_sample_rate / 2, 5e5**np.fix(np.log(gen_sample_rate / 2) / np.log(5e5)))) * 1e-6
    plt.xticks(np.hstack([np.arange(-closest_500_khz_divisor, closest_500_khz_divisor + 0.1, step=0.5), np.array([-gen_sample_rate / 2 * 1e-6, gen_sample_rate / 2 * 1e-6])]))
    plt.grid()

In [ ]:
pss_corr_mags = np.vstack([ np.abs(signal.correlate(obsrv_samples, np.asarray(pss_waveform), mode="valid", method="fft")) for pss_waveform in pss_waveforms ])
pss_x = np.arange(pss_corr_mags.shape[1]) * 1000 / gen_sample_rate

pss_peak_idx = np.argmax(pss_corr_mags, axis=1)
pss_peak_time_offset = pss_peak_idx / gen_sample_rate

print(f"Correlation peaks at {np.array2string(pss_peak_idx)} with time offset of {np.array2string(pss_peak_time_offset * 1000, precision=3)} msec")

In [ ]:
for idx, corr_mag in enumerate(pss_corr_mags[:]):
    corr_mag_norm = corr_mag / np.max(corr_mag)
    plt.figure(figsize=(35, 3))
    plt.plot(pss_x, corr_mag_norm)
    plt.title(f"Correlation of Generated PSS {idx} with Observation Signal")
    plt.xlabel("time delta [msec]")
    plt.ylabel("correlation (norm.)")
    plt.xticks(np.linspace(0, T_FRAME * frame_count * 1000, frame_count * 2 + 1))
    plt.grid()

    plt.annotate(f"Peak\n"
                f"sample offset: {pss_peak_idx[idx]}\n"
                f"time offset: {pss_peak_time_offset[idx] * 1000:0.3f} msec",
                xy=(pss_peak_time_offset[idx] * 1000, corr_mag_norm[pss_peak_idx[idx]]),
                xytext=(50, 40),
                xycoords="data",
                textcoords="offset pixels",
                arrowprops={'arrowstyle': 'wedge'})

    if pss_peak_time_offset[idx] + T_FRAME * 4 < frame_count * T_FRAME:
        plt.axvspan((pss_peak_time_offset[idx] + T_FRAME * 4) * 1000 - 0.5, (pss_peak_time_offset[idx] + T_FRAME * 4) * 1000 + 0.5, color='red', alpha=0.4)
        plt.annotate(f"Estimated next PSS\n"
                    f"time offset: {(pss_peak_time_offset[idx] + T_FRAME * 4) * 1000:0.3f} msec",
                    xy=((pss_peak_time_offset[idx] + T_FRAME * 4) * 1000, 1),
                    xytext=(-100, 40),
                    xycoords="data",
                    textcoords="offset pixels",
                    arrowprops={'arrowstyle': 'wedge'})

    if pss_peak_time_offset[idx] - T_FRAME * 4 > 0:
        plt.axvspan((pss_peak_time_offset[idx] - T_FRAME * 4) * 1000 - 0.5, (pss_peak_time_offset[idx] - T_FRAME * 4) * 1000 + 0.5, color='red', alpha=0.4)
        plt.annotate(f"Estimated previous PSS\n"
                    f"time offset: {(pss_peak_time_offset[idx] - T_FRAME * 4) * 1000:0.3f} msec",
                    xy=((pss_peak_time_offset[idx] - T_FRAME * 4) * 1000, 1),
                    xytext=(-100, 40),
                    xycoords="data",
                    textcoords="offset pixels",
                    arrowprops={'arrowstyle': 'wedge'})


In [ ]:
max_corr_pss_idx = np.argmax(np.diag(pss_corr_mags[:, pss_peak_idx])).item()
cell_id_in_group = max_corr_pss_idx

print(f"Highest correlation with PSS index {max_corr_pss_idx}, correlation value: {pss_corr_mags[max_corr_pss_idx, pss_peak_idx[max_corr_pss_idx]]:.3f}")

## SSS Synchronization

After obtaining a fix on the PSS we can now try to decode the SSS. The SSS is an scrambled interleaved concatination of two 31 bit sequences.


In [ ]:
d = pluto_sdr_pr.signal.generate_sss_sequence(cell_id_in_group, 0)

Similar to PSS we now construct several reference subframes containing every possible SSS.


In [ ]:
grid = np.zeros((d.shape[0], num_dl_sc, symbols_per_subframe), dtype=np.complex128)

grid[:, np.arange(sss_sc_offset_in_symbol, sss_sc_offset_in_symbol + d.shape[1]), sss_symbol_idx_in_slot] = d[:, :]

sss_waveforms, gen_sample_rate = pluto_sdr_pr.signal.ofdm_modulate_subframe(grid)

In [ ]:
if NO_CUPY:
    sss_corr_mags = np.vstack([ np.abs(signal.correlate(obsrv_samples, sss_waveform, mode="valid", method="fft")) for sss_waveform in sss_waveforms ])
else:
    sss_corr_mags = np.vstack([ cp.abs(cp.correlate(cp.asarray(obsrv_samples), cp.asarray(sss_waveform), mode="valid")).get() for sss_waveform in sss_waveforms ])
sss_x = np.arange(sss_corr_mags.shape[1]) * 1000 / gen_sample_rate

sss_peak_idx = np.argmax(sss_corr_mags, axis=1)
sss_peak_time_offset = sss_peak_idx / gen_sample_rate

print(f"Correlation peaks at {np.array2string(sss_peak_idx)} with time offset of {np.array2string(sss_peak_time_offset * 1000, precision=3)} msec")

In [ ]:
max_corr_sss_idx = np.argmax(np.diag(sss_corr_mags[:, sss_peak_idx])).item()
cell_id_group = max_corr_sss_idx

print(f"Highest correlation with SSS index {max_corr_sss_idx}, correlation value: {sss_corr_mags[max_corr_sss_idx, sss_peak_idx[max_corr_sss_idx]]:.3f}")

To visually verify the above results, plot the correlation of the highest scoring SSS index.


In [ ]:
corr_mag_norm = sss_corr_mags[max_corr_sss_idx] / np.max(sss_corr_mags[max_corr_sss_idx])
plt.figure(figsize=(35, 3))
plt.plot(sss_x, corr_mag_norm)
plt.title(f"Correlation of Generated SSS {max_corr_sss_idx} with Observation Signal")
plt.xlabel("time delta [msec]")
plt.ylabel("correlation (norm.)")
plt.xticks(np.linspace(0, T_FRAME * frame_count * 1000, frame_count * 2 + 1))
plt.grid()

plt.annotate(f"Peak\n"
            f"sample offset: {sss_peak_idx[max_corr_sss_idx]}\n"
            f"time offset: {sss_peak_time_offset[max_corr_sss_idx] * 1000:0.3f} msec",
            xy=(sss_peak_time_offset[max_corr_sss_idx] * 1000, corr_mag_norm[sss_peak_idx[max_corr_sss_idx]]),
            xytext=(50, 40),
            xycoords="data",
            textcoords="offset pixels",
            arrowprops={'arrowstyle': 'wedge'});

## Determine Cell ID

After determining PSS and SSS we can calculate a locally-unique cell identity. That is, one of 504 different cell identites, which are assigned in a way as to avoid neighbouring cells to have the same identity.


In [ ]:
cell_id = 3 * cell_id_group + cell_id_in_group
print(f"Cell ID resulting from PSS index {max_corr_pss_idx} and SSS index {max_corr_sss_idx}: {cell_id}")

samples_per_frame = gen_sample_rate / 100
skew_factor = gen_sample_rate / sample_rate
pss_offset = pss_peak_idx[max_corr_pss_idx] % (samples_per_frame)

In [ ]:
pss_max_corr_mag = pss_corr_mags[max_corr_pss_idx] / np.max(pss_corr_mags[max_corr_pss_idx])
plt.figure(figsize=(35, 3))
plt.plot(pss_max_corr_mag)
plt.title(f"Correlation of Generated PSS {max_corr_pss_idx} with Observation Signal")
plt.xlabel("samples")
plt.ylabel("correlation (norm.)")
num_fft_bins = pluto_sdr_pr.signal.get_num_fft_bins(num_dl_sc)
cyc_prefix_lengths = pluto_sdr_pr.signal.get_cyclic_prefix_lengths(num_fft_bins)
start_of_frame = 768000 - pss_symbol_idx_in_slot * num_fft_bins + np.sum(cyc_prefix_lengths[:pss_symbol_idx_in_slot])
plt.grid()

peaks = np.argwhere(pss_max_corr_mag > 0.6)[:, 0]
peaks = np.append(peaks[np.argwhere(np.diff(peaks, axis=0) > 2)], peaks[-1])
print(peaks, np.diff(np.diff(peaks)))